## Plotting a Hurling Match

### Import and Roughly Clean Data

In [1]:
pip list

Package                       Version
----------------------------- ------------------
absl-py                       1.3.0
antlr4-python3-runtime        4.9.3
appnope                       0.1.3
astor                         0.8.1
attrs                         22.2.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
black                         22.12.0
Bottleneck                    1.3.5
brotlipy                      0.7.0
certifi                       2022.12.7
cffi                          1.15.1
chardet                       5.1.0
charset-normalizer            3.0.1
click                         8.0.4
cloudpickle                   2.2.1
cryptography                  38.0.4
cycler                        0.11.0
Cython                        0.29.33
dataclasses                   0.6
debugpy                       1.5.1
decorator                     4.4.2
Deprecated                    1.2.13
emoji                         2.2.0
entrypoints                   0.4
ffmp

In [ ]:
# The Sliotar is too small to detect, so let's identify significant events and fill in the gaps from there
# Rebuild video with frame numbers to make events easier to identify
import math

file_path = "".join(path.split(".")[:-1]).replace("labels/", "") + ".mov"  # change to your own video path
# print(file_path)
cap = cv2.VideoCapture(file_path)
vid_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
vid_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# print(vid_height, vid_width)

video=cv2.VideoWriter(("".join(path.split(".")[:-1]).replace("labels/", "") + "_plusframes.mp4") ,cv2.VideoWriter_fourcc(*'mp4v'),out_fps,(vid_width,vid_height))
  
x = 3
while(True):
    ret, frame = cap.read()
    if ret == True:
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, 
                    str(x), 
                    (50, 50), 
                    font, 1, 
                    (0, 255, 255), 
                    2, 
                    cv2.LINE_4)
    
        x += 1
        video.write(frame)
    else:
        break
cv2.destroyAllWindows()
video.release()

In [2]:
import pandas as pd
import cv2
import numpy as np

out_fps = 29.3

run = 16
player17h = f"exp{run}/labels/Hurling_intersquad_passes.txt"
df = pd.read_csv(player17h, header=None, sep=" ", names=["frame", "class", "ID", "bb_left", "bb_top", "bb_w", "bb_h", "None1", "None2", "None3", "None4", "NaN"])
df['x_mid'] = df['bb_left'] + (df['bb_w'] / 2)
df['y_mid'] = df['bb_top'] + (df['bb_h'] / 2)
df = df[['frame', 'class', 'ID', 'x_mid', 'y_mid']].copy()
df = df.loc[(df['class']==0)] #Person & No Sports Ball

In [3]:
df.shape

(40487, 5)

In [4]:
df['frame'].nunique()

1959

### Identify Players

In [5]:
for frame in df['frame'].unique():
    print(frame, df[(df['frame'] == frame) & (df['class'] == 0)]['ID'].unique())

3 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
4 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
5 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
6 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
7 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
8 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
9 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
10 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
11 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
12 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
13 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
14 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
15 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
16 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
17 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
18 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
19 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
20 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
21 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
22 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
23 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 16]
24 [ 1  2  3  4  5  

In [6]:
players = {1:"player1",
           2:"player2",
           3:"player3",
           4:"player4",
           5:"player5",
            6:"player6",
            7:"player7",
            8:"Referee",
            9:"Eric",
            10:"player8",
            11:"player9",
            12:"player10",
            13:"player11",
            16:"player12",
            47:"player13",
            83:"player14",
            94:"player15",
            99:"player16",
            128:"player17",
            138:"player18",
            148:"player19",
            180:"player20",
            190:"player21",
            191:"player8",
            199:"player22",
            203:"player22",
            226:"player1",
            266:"player19",
            285:"player14",
            334:"player13",
            320:"player17",
            375:"player17",
            454:"player1",
            455:"player10",
            516:"player7",
            533:"player7",
            555:"player7",
            571:"player9",
           }
team = {1:"F",
        2:"R",
        3:"H",
        4:"H",
        5:"H",
        6:"F",
        7:"H",
        8:"R",
        9:"H",
        10:"F",
        11:"F",
        12:"F",
        13:"H",
        16:"F",
        47:"F",
        83:"F",
        94:"F",
        99:"H",
        128:"H",
        138:"F",
        148:"F",
        180:"H",
        190:"H",
        191:"F",
        199:"H",
        203:"H",
        226:"F",
        266:"F",
        285:"F",
        334:"F",
        320:"H",
        375:"H",
        454:"F",
        455:"F",
        516:"H",
        533:"H",
        555:"H",
        571:"F",
        }
players = pd.DataFrame.from_dict(players, orient='index', columns=['Name']).reset_index().rename(columns={"index":"ID"})
team = pd.DataFrame.from_dict(team, orient='index', columns=['Team']).reset_index().rename(columns={"index":"ID"})

#### Clean Players

In [7]:
# Dropping short-lived & non-person IDs
df = df.drop(df[(df['ID'].isin([44,75,96,437,485,463,570,705,711]))].index)
# 516 not needed bc it flips between player7 and player1, plus they're not moving
df = df.drop(df[(df['ID'].isin([516]))].index)

In [8]:
df = pd.merge(df, players, how='left', on='ID')

In [9]:
df = pd.merge(df, team, how='left', on='ID')

In [10]:
#16 becomes player18 after ID 138 and team becomes F
df.loc[(df['frame'] > df[df['ID']==138]['frame'].max()) & (df['ID'] == 16), "Name"] = "player18"
df.loc[(df['frame'] > df[df['ID']==138]['frame'].max()) & (df['ID'] == 16), "Team"] = "F"

#Drop ID 94 at 403
df = df.drop(df[(df['frame'] == 403) & (df['ID'].isin([94]))].index)

#player1 and player18 switch 403. player1 is 16, player18 is 1, both are F
df.loc[(df['frame'] >= 403) & (df['ID'] == 1), "Name"] = "player18"
df.loc[(df['frame'] >= 403) & (df['ID'] == 16), "Name"] = "player1"

#468: player1 is 1, player15 is 16, player18 is 94, all are F
df.loc[(df['frame'] >= 468) & (df['ID'] == 1), "Name"] = "player1"
df.loc[(df['frame'] >= 468) & (df['ID'] == 16), "Name"] = "player15"
df.loc[(df['frame'] >= 468) & (df['ID'] == 94), "Name"] = "player18"

In [11]:
df

,frame,class,ID,x_mid,y_mid,Name,Team
0,3,0,1,3490.5,720.0,player1,F
1,3,0,2,1994.0,1141.5,player2,R
2,3,0,3,880.5,494.5,player3,H
3,3,0,4,2582.0,1034.5,player4,H
4,3,0,5,1965.0,483.0,player5,H
...,...,...,...,...,...,...,...
40381,1961,0,334,533.5,430.0,player13,F
40382,1961,0,375,85.0,506.5,player17,H
40383,1961,0,454,3207.5,800.0,player1,F
40384,1961,0,455,1070.0,453.0,player10,F


In [12]:
clean = df[['frame', 'Name']].value_counts()
clean.reset_index()[clean.reset_index()[0]>1].sort_values(by='frame')

,frame,Name,0
4,458,player8,2
3,1202,player10,2
1,1203,player10,2
2,1204,player10,2
0,1206,player10,2


In [13]:
df = df.drop(df[(df['frame'] == 458) & (df['ID'].isin([191]))].index)
df = df.drop(df[(df['frame'] == 1202) & (df['ID'].isin([455]))].index)
df = df.drop(df[(df['frame'] == 1203) & (df['ID'].isin([455]))].index)
df = df.drop(df[(df['frame'] == 1204) & (df['ID'].isin([455]))].index)
df = df.drop(df[(df['frame'] == 1206) & (df['ID'].isin([455]))].index)

In [14]:
clean = df[['frame', 'Name']].value_counts()
clean.reset_index()[clean.reset_index()[0]>1].sort_values(by='frame')

,frame,Name,0


### Check any remaining NANs

In [15]:
df[df.isna().any(axis=1)]

,frame,class,ID,x_mid,y_mid,Name,Team
825,61,0,15,1427.0,213.0,NaN,NaN
1564,111,0,15,1430.0,216.5,NaN,NaN
22838,1159,0,361,1463.0,191.5,NaN,NaN
22860,1160,0,361,1409.0,189.5,NaN,NaN
22968,1165,0,361,1462.5,189.5,NaN,NaN
22991,1166,0,361,1462.5,189.5,NaN,NaN
23014,1167,0,361,1462.0,189.5,NaN,NaN
23037,1168,0,361,1461.5,189.5,NaN,NaN
23060,1169,0,361,1461.5,190.0,NaN,NaN
23083,1170,0,361,1461.5,190.0,NaN,NaN


In [16]:
df = df.drop(df[(df['ID'].isin([14, 15, 361,462, 536,548,561]))].index)

In [17]:
df[df.isna().any(axis=1)]

,frame,class,ID,x_mid,y_mid,Name,Team


### Identify Ball

In [18]:
ball_actions = {3:"player12",
                20:"Pass",
                73:"player6",
                76:"Lost",
                 129: "Eric",
                 132:"Pass",
                 153:"player5",
                189:"Pass",
                268:"player15",
                277:"Lost",
                435:"player18",
                542:"Pass",
                650:"player10",
                731:"Pass",
                818:"player13",
                819:"Lost",
                859:"player17",
                910:"Pass",
                933:"player11",
                986:"Pass",
                1040:"player5",
                1041:"Lost",
                1173:"player8",
                1227:"Pass",
                1258:"player10",
                1350:"Pass",
                1403:"player13",
                1448:"Pass",
                1492:"Eric",
                1494:"Lost",
                1547:"Eric",
                1617:"Pass",
                1740:"player14",
                1741:"Lost",
                1781:"player4",
                1782:"Pass",
                1885:"player7",
                1919:"Pass",
                1943:"player20",
                }
ball_actions = pd.DataFrame.from_dict(ball_actions, orient="index",columns=['action']).reset_index().rename(columns={"index":"frame"})

In [19]:
ball_actions#['Name'].map({"player12":"H"})

,frame,action
0,3,player12
1,20,Pass
2,73,player6
3,76,Lost
4,129,Eric
5,132,Pass
6,153,player5
7,189,Pass
8,268,player15
9,277,Lost


In [20]:
ball = []
for frame in list(df.frame.unique()):
    if ball_actions[ball_actions["frame"] == ball_actions[ball_actions['frame'] <= frame]['frame'].max()]['action'].values:
        name = ball_actions[ball_actions["frame"] == ball_actions[ball_actions['frame'] <= frame]['frame'].max()]['action'].values[0]
        if df[(df['frame'] == frame) & (df['Name'] == name)].shape[0] == 1:
            ball_df = df[(df['frame'] == frame) & (df['Name'] == name)].copy()
            ball_df['Name'] = "ball"
            ball_df['Team'] = "N"
            ball_df['ID'] = 0
            ball_df['class'] = 32
            ball.append(ball_df.copy())
        else:
            print(frame, "Issue 1") #Ball being passed/lost
    else:
        print(frame, "Issue 2")
ball = pd.concat(ball)

20 Issue 1
21 Issue 1
22 Issue 1
23 Issue 1
24 Issue 1
25 Issue 1
26 Issue 1
27 Issue 1
28 Issue 1
29 Issue 1
30 Issue 1
31 Issue 1
32 Issue 1
33 Issue 1
34 Issue 1
35 Issue 1
36 Issue 1
37 Issue 1
38 Issue 1
39 Issue 1
40 Issue 1
41 Issue 1
42 Issue 1
43 Issue 1
44 Issue 1
45 Issue 1
46 Issue 1
47 Issue 1
48 Issue 1
49 Issue 1
50 Issue 1
51 Issue 1
52 Issue 1
53 Issue 1
54 Issue 1
55 Issue 1
56 Issue 1
57 Issue 1
58 Issue 1
59 Issue 1
60 Issue 1
61 Issue 1
62 Issue 1
63 Issue 1
64 Issue 1
65 Issue 1
66 Issue 1
67 Issue 1
68 Issue 1
69 Issue 1
70 Issue 1
71 Issue 1
72 Issue 1
76 Issue 1
77 Issue 1
78 Issue 1
79 Issue 1
80 Issue 1
81 Issue 1
82 Issue 1
83 Issue 1
84 Issue 1
85 Issue 1
86 Issue 1
87 Issue 1
88 Issue 1
89 Issue 1
90 Issue 1
91 Issue 1
92 Issue 1
93 Issue 1
94 Issue 1
95 Issue 1
96 Issue 1
97 Issue 1
98 Issue 1
99 Issue 1
100 Issue 1
101 Issue 1
102 Issue 1
103 Issue 1
104 Issue 1
105 Issue 1
106 Issue 1
107 Issue 1
108 Issue 1
109 Issue 1
110 Issue 1
111 Issue 1
112 Issue

In [21]:
df = pd.concat([df,ball])

In [22]:
df = df.sort_values("frame").reset_index(drop=True)

### Interpolate Locations

In [23]:
df

,frame,class,ID,x_mid,y_mid,Name,Team
0,3,0,1,3490.5,720.0,player1,F
1,3,0,16,3806.5,583.5,player12,F
2,3,0,13,423.5,1184.5,player11,H
3,3,0,12,2237.0,343.5,player10,F
4,3,0,11,922.0,504.5,player9,F
...,...,...,...,...,...,...,...
40949,1961,0,8,1977.5,553.0,Referee,R
40950,1961,0,6,1749.5,1087.5,player6,F
40951,1961,0,5,1351.0,427.5,player5,H
40952,1961,0,99,3157.5,467.0,player16,H


In [24]:
outer_df = []
for frame in df['frame'].unique():
    for name in list(players.Name.unique()) + ['ball']:
        outer_df.append({"frame":frame, "Name":name})
outer_df = pd.DataFrame(outer_df)

In [25]:
outer_df

,frame,Name
0,3,player1
1,3,player2
2,3,player3
3,3,player4
4,3,player5
...,...,...
48970,1961,player19
48971,1961,player20
48972,1961,player21
48973,1961,player22


In [26]:
df = pd.merge(df,outer_df, on=['frame', 'Name'], how='outer')

In [27]:
df = df.sort_values(['frame', 'Name']).reset_index(drop=True)

In [28]:
df[df['Name']=="ball"].head(50)

,frame,class,ID,x_mid,y_mid,Name,Team
2,3,32.0,0.0,3806.5,583.5,ball,N
27,4,32.0,0.0,3798.5,582.0,ball,N
52,5,32.0,0.0,3790.0,578.5,ball,N
77,6,32.0,0.0,3779.0,576.5,ball,N
102,7,32.0,0.0,3775.0,576.5,ball,N
127,8,32.0,0.0,3767.5,577.5,ball,N
152,9,32.0,0.0,3761.0,577.0,ball,N
177,10,32.0,0.0,3745.0,576.0,ball,N
202,11,32.0,0.0,3735.0,575.5,ball,N
227,12,32.0,0.0,3731.5,574.0,ball,N


In [29]:
outdf = []

for Name, group in df.groupby("Name"):
    group['x_mid'] = group['x_mid'].interpolate()
    group['y_mid'] = group['y_mid'].interpolate()
    outdf.append(group)
df = pd.concat(outdf)

In [30]:
df[df['Name']=="ball"].head(50)

,frame,class,ID,x_mid,y_mid,Name,Team
2,3,32.0,0.0,3806.500000,583.500000,ball,N
27,4,32.0,0.0,3798.500000,582.000000,ball,N
52,5,32.0,0.0,3790.000000,578.500000,ball,N
77,6,32.0,0.0,3779.000000,576.500000,ball,N
102,7,32.0,0.0,3775.000000,576.500000,ball,N
127,8,32.0,0.0,3767.500000,577.500000,ball,N
152,9,32.0,0.0,3761.000000,577.000000,ball,N
177,10,32.0,0.0,3745.000000,576.000000,ball,N
202,11,32.0,0.0,3735.000000,575.500000,ball,N
227,12,32.0,0.0,3731.500000,574.000000,ball,N


In [31]:
df.columns

Index(['frame', 'class', 'ID', 'x_mid', 'y_mid', 'Name', 'Team'], dtype='object')

In [32]:
name_team = df[['Name', "Team"]].drop_duplicates().dropna()

In [33]:
df.drop(columns=['Team'], inplace=True)
df = pd.merge(df, name_team, on=['Name'], how='left')

In [34]:
df[df['Name']=="ball"].head(50)

,frame,class,ID,x_mid,y_mid,Name,Team
3918,3,32.0,0.0,3806.500000,583.500000,ball,N
3919,4,32.0,0.0,3798.500000,582.000000,ball,N
3920,5,32.0,0.0,3790.000000,578.500000,ball,N
3921,6,32.0,0.0,3779.000000,576.500000,ball,N
3922,7,32.0,0.0,3775.000000,576.500000,ball,N
3923,8,32.0,0.0,3767.500000,577.500000,ball,N
3924,9,32.0,0.0,3761.000000,577.000000,ball,N
3925,10,32.0,0.0,3745.000000,576.000000,ball,N
3926,11,32.0,0.0,3735.000000,575.500000,ball,N
3927,12,32.0,0.0,3731.500000,574.000000,ball,N


In [35]:
df = df.drop(columns=['class', 'ID'])

In [36]:
# player12 goes out of frame after 230, so let's remove his out of frame records
df = df.drop(df[(df['frame'] >= 230) & (df['Name'] == "player12")].index)
# player14 goes off screen at 281 and comes back at 820
df = df.drop(df[(df['frame'] >= 281) & (df['frame'] <= 820) & (df['Name'] == "player14")].index)
# player19 goes off screen at 352 and comes back at 793
df = df.drop(df[(df['frame'] >= 352) & (df['frame'] <= 793)  & (df['Name'] == "player19")].index)
# player1 goes off-screen from 490-572
df = df.drop(df[(df['frame'] >= 490) & (df['frame'] <= 572)  & (df['Name'] == "player1")].index)
# player9 goes off screen around 1670
df = df.drop(df[(df['frame'] >= 1670) & (df['Name'] == "player9")].index)
# player3 goes off screen around 1713
df = df.drop(df[(df['frame'] >= 1713) & (df['Name'] == "player3")].index)
# player17 & player13 go offscreen from ~900-1000 but there's a pass I want to track and they obviously don't go far
# We'll stick with those interpolated values

### Begin Calculations

In [37]:
quad_coords = {
    "lonlat": np.array([
        [41.708893, -72.585288], # Top Right Soccer Corner
        [41.709869, -72.585478], # Top left Soccer Corner
        [41.709476, -72.586003], # Bottom Left
        [41.709113, -72.586045] # Bottom Right
    ]),
    "pixel": np.array([
        [3725, 290], # top right soccer corner
        [31, 271], # top left
        [25, 2118], # bottom left
        [3760, 2135] # bottom right
    ])
}

In [38]:
import numpy as np
import cv2


class PixelMapper(object):
    """
    Create an object for converting pixels to geographic coordinates,
    using four points with known locations which form a quadrilteral in both planes
    Parameters
    ----------
    pixel_array : (4,2) shape numpy array
        The (x,y) pixel coordinates corresponding to the top left, top right, bottom right, bottom left
        pixels of the known region
    lonlat_array : (4,2) shape numpy array
        The (lon, lat) coordinates corresponding to the top left, top right, bottom right, bottom left
        pixels of the known region
    """
    def __init__(self, pixel_array, lonlat_array):
        assert pixel_array.shape==(4,2), "Need (4,2) input array"
        assert lonlat_array.shape==(4,2), "Need (4,2) input array"
        self.M = cv2.getPerspectiveTransform(np.float32(pixel_array),np.float32(lonlat_array))
        self.invM = cv2.getPerspectiveTransform(np.float32(lonlat_array),np.float32(pixel_array))
        
    def pixel_to_lonlat(self, pixel):
        """
        Convert a set of pixel coordinates to lon-lat coordinates
        Parameters
        ----------
        pixel : (N,2) numpy array or (x,y) tuple
            The (x,y) pixel coordinates to be converted
        Returns
        -------
        (N,2) numpy array
            The corresponding (lon, lat) coordinates
        """
        if type(pixel) != np.ndarray:
            pixel = np.array(pixel).reshape(1,2)
        assert pixel.shape[1]==2, "Need (N,2) input array" 
        pixel = np.concatenate([pixel, np.ones((pixel.shape[0],1))], axis=1)
        lonlat = np.dot(self.M,pixel.T)
        
        return (lonlat[:2,:]/lonlat[2,:]).T
    
    def lonlat_to_pixel(self, lonlat):
        """
        Convert a set of lon-lat coordinates to pixel coordinates
        Parameters
        ----------
        lonlat : (N,2) numpy array or (x,y) tuple
            The (lon,lat) coordinates to be converted
        Returns
        -------
        (N,2) numpy array
            The corresponding (x, y) pixel coordinates
        """
        if type(lonlat) != np.ndarray:
            lonlat = np.array(lonlat).reshape(1,2)
        assert lonlat.shape[1]==2, "Need (N,2) input array" 
        lonlat = np.concatenate([lonlat, np.ones((lonlat.shape[0],1))], axis=1)
        pixel = np.dot(self.invM,lonlat.T)
        
        return (pixel[:2,:]/pixel[2,:]).T

In [39]:
pm = PixelMapper(quad_coords["pixel"], quad_coords["lonlat"])

In [40]:
df['lon_lat'] = df.apply(lambda x: pm.pixel_to_lonlat((x['x_mid'], x['y_mid']))[0], axis=1)

In [41]:
df['lon'] = df['lon_lat'].str[0]
df['lat'] = df['lon_lat'].str[1]

In [42]:
import haversine as hs
from haversine import Unit

outdf = []
for Name, group in df.groupby("Name"):
    print(Name)
    group['lon_prev'] = group['lon'].shift(1)
    group['lat_prev'] = group['lat'].shift(1)
    group['haversine'] = group.apply(lambda x: hs.haversine((x['lat'], x['lon']), (x['lat_prev'], x['lon_prev']), unit=Unit.MILES), axis=1)
    group['haversine_smooth'] = group['haversine'].rolling(window=5).mean()
    group['haversine_smooth'] = group['haversine'].rolling(window=29).mean()
    outdf.append(group)
df = pd.concat(outdf).sort_index()

Eric
Referee
ball
player1
player10
player11
player12
player13
player14
player15
player16
player17
player18
player19
player2
player20
player21
player22
player3
player4
player5
player6
player7
player8
player9


#### Derive Insights

In [43]:
df['mph'] = df['haversine'] * 29 * 3600
df['mph_smoothed'] = df['haversine_smooth'] * 29 * 3600

In [44]:
df.dropna()[~df['Name'].isin(["ball"])].sort_values('mph').tail(50) #[~df['Name'].isin(["player6", "player13", "player12"])] .dropna()[~df['Name'].isin(["ball"])]

/Users/ericstassen/opt/anaconda3/envs/vision37/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed
40142,965,1598.000000,369.500000,player5,H,"[41.70953456775842, -72.58549529263706]",41.709535,-72.585495,41.709530,-72.585505,0.000703,0.000090,73.351720,9.390741
20281,694,3062.000000,399.000000,player16,H,"[41.709154757510525, -72.58545901887584]",41.709155,-72.585459,41.709152,-72.585449,0.000705,0.000148,73.550206,15.496641
23967,462,3772.625000,543.250000,player18,F,"[41.708957636967476, -72.58555422954574]",41.708958,-72.585554,41.708963,-72.585564,0.000707,0.000092,73.819983,9.589019
23968,463,3778.500000,530.500000,player18,F,"[41.708952511095895, -72.58554383213163]",41.708953,-72.585544,41.708958,-72.585554,0.000726,0.000117,75.812174,12.203232
41733,597,2988.500000,556.500000,player6,F,"[41.70918197063027, -72.58558683365702]",41.709182,-72.585587,41.709182,-72.585598,0.000760,0.000062,79.375763,6.457146
23970,465,3705.000000,538.000000,player18,F,"[41.70897751613349, -72.58555232317264]",41.708978,-72.585552,41.708952,-72.585544,0.000778,0.000143,81.243909,14.918168
43498,403,3518.000000,507.000000,player7,H,"[41.70902880414296, -72.58553395755983]",41.709029,-72.585534,41.709029,-72.585545,0.000780,0.000092,81.383874,9.569871
501,504,1891.500000,427.000000,Eric,H,"[41.70945602520689, -72.58552791019513]",41.709456,-72.585528,41.709452,-72.585540,0.000813,0.000283,84.831052,29.565369
1488,1491,504.000000,306.000000,Eric,H,"[41.709772383835706, -72.58548739484552]",41.709772,-72.585487,41.709780,-72.585476,0.000820,0.000112,85.611043,11.640680
462,465,1920.500000,450.500000,Eric,H,"[41.7094448118957, -72.5855436820465]",41.709445,-72.585544,41.709449,-72.585532,0.000831,0.000064,86.740413,6.694943


In [45]:
df.dropna()[df['Name'].isin(["ball"])].sort_values('mph').tail(50)

/Users/ericstassen/opt/anaconda3/envs/vision37/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed
4667,752,2250.375000,394.750000,ball,N,"[41.7093778935752, -72.58549044329258]",41.709378,-72.585490,41.709371,-72.585487,0.000280,0.000254,29.204399,26.483757
5536,1621,645.411290,451.197581,ball,N,"[41.70968898115515, -72.58557746391934]",41.709689,-72.585577,41.709692,-72.585574,0.000282,0.000118,29.434600,12.299275
5789,1874,3010.552885,752.764423,ball,N,"[41.70918178189115, -72.58570026453036]",41.709182,-72.585700,41.709186,-72.585704,0.000283,0.000234,29.513863,24.417814
4666,751,2277.948864,391.715909,ball,N,"[41.70937148985099, -72.5854868769817]",41.709371,-72.585487,41.709365,-72.585483,0.000283,0.000248,29.573200,25.933508
5535,1620,640.629032,444.258065,ball,N,"[41.70969227317908, -72.58557350407564]",41.709692,-72.585574,41.709696,-72.585570,0.000285,0.000115,29.715870,12.026163
4665,750,2305.522727,388.681818,ball,N,"[41.70936500525916, -72.58548326563466]",41.709365,-72.585483,41.709358,-72.585480,0.000287,0.000242,29.949031,25.216223
5790,1875,3029.048077,745.240385,ball,N,"[41.70917752262518, -72.5856963114954]",41.709178,-72.585696,41.709182,-72.585700,0.000287,0.000237,29.960493,24.755226
5534,1619,635.846774,437.318548,ball,N,"[41.70969559666077, -72.58556950639264]",41.709696,-72.585570,41.709699,-72.585565,0.000287,0.000107,30.001191,11.162665
5533,1618,631.064516,430.379032,ball,N,"[41.709698952053316, -72.58556547032536]",41.709699,-72.585565,41.709702,-72.585561,0.000290,0.000098,30.290642,10.278871
4664,749,2333.096591,385.647727,ball,N,"[41.70935843825815, -72.58547960839296]",41.709358,-72.585480,41.709352,-72.585476,0.000291,0.000234,30.332072,24.477520


### Player Data was messier so let's clean up obvious errors

In [46]:
# The ball doesn't appear to be going over 50mph horizontally on it's own. 
# People probably arent moving about 30mph, but that can be allowed for a single frame.
df.loc[(df['mph'] > 50), 'x_mid'] = np.nan
df.loc[(df['mph'] > 50), 'y_mid'] = np.nan

# OC's data is a little messier, let's interpolate entirely from 460-480
df.loc[(df['frame'] > 460) & (df['frame'] < 480) & (df['Name'] == "player15"), 'x_mid'] = np.nan
df.loc[(df['frame'] > 460) & (df['frame'] < 480) & (df['Name'] == "player15"), 'y_mid'] = np.nan

# player18's data is a little messier, let's interpolate entirely from 458-505
df.loc[(df['frame'] > 458) & (df['frame'] < 505) & (df['Name'] == "player18"), 'x_mid'] = np.nan
df.loc[(df['frame'] > 458) & (df['frame'] < 505) & (df['Name'] == "player18"), 'y_mid'] = np.nan
# player18 has the ball, let's interpolate entirely from 458-505
df.loc[(df['frame'] > 458) & (df['frame'] < 505) & (df['Name'] == "ball"), 'x_mid'] = np.nan
df.loc[(df['frame'] > 458) & (df['frame'] < 505) & (df['Name'] == "ball"), 'y_mid'] = np.nan

# player18's data is a little messier, let's interpolate entirely from 340-440
df.loc[(df['frame'] > 340) & (df['frame'] < 440) & (df['Name'] == "player18"), 'x_mid'] = np.nan
df.loc[(df['frame'] > 340) & (df['frame'] < 440) & (df['Name'] == "player18"), 'y_mid'] = np.nan

# player1's data is a little messier, let's interpolate entirely from 340-440
df.loc[(df['frame'] > 340) & (df['frame'] < 440) & (df['Name'] == "player1"), 'x_mid'] = np.nan
df.loc[(df['frame'] > 340) & (df['frame'] < 440) & (df['Name'] == "player1"), 'y_mid'] = np.nan

# player1's data is a little messier, let's interpolate entirely from 574-600
df.loc[(df['frame'] > 574) & (df['frame'] < 600) & (df['Name'] == "player1"), 'x_mid'] = np.nan
df.loc[(df['frame'] > 574) & (df['frame'] < 600) & (df['Name'] == "player1"), 'y_mid'] = np.nan
# player1's data is a little messier, let's interpolate entirely from 470-489
df.loc[(df['frame'] > 470) & (df['frame'] < 489) & (df['Name'] == "player1"), 'x_mid'] = np.nan
df.loc[(df['frame'] > 470) & (df['frame'] < 489) & (df['Name'] == "player1"), 'y_mid'] = np.nan

# player19's data is a little messier, let's interpolate entirely from 793-850
df.loc[(df['frame'] > 793) & (df['frame'] < 850) & (df['Name'] == "player19"), 'x_mid'] = np.nan
df.loc[(df['frame'] > 793) & (df['frame'] < 850) & (df['Name'] == "player19"), 'y_mid'] = np.nan

In [47]:
outdf = []

for Name, group in df.groupby("Name"):
    group['x_mid'] = group['x_mid'].interpolate()
    group['y_mid'] = group['y_mid'].interpolate()
    outdf.append(group)
df = pd.concat(outdf)

In [48]:
df['lon_lat'] = df.apply(lambda x: pm.pixel_to_lonlat((x['x_mid'], x['y_mid']))[0], axis=1)
df['lon'] = df['lon_lat'].str[0]
df['lat'] = df['lon_lat'].str[1]
outdf = []
for Name, group in df.groupby("Name"):
    print(Name)
    group['lon_prev'] = group['lon'].shift(1)
    group['lat_prev'] = group['lat'].shift(1)
    group['haversine'] = group.apply(lambda x: hs.haversine((x['lat'], x['lon']), (x['lat_prev'], x['lon_prev']), unit=Unit.MILES), axis=1)
    group['haversine_smooth_10'] = group['haversine'].rolling(window=10).mean()
    group['haversine_smooth'] = group['haversine'].rolling(window=29).mean()
    outdf.append(group)
df = pd.concat(outdf).sort_index()
df['mph'] = df['haversine'] * 29 * 3600
df['mph_smoothed_10'] = df['haversine_smooth_10'] * 29 * 3600
df['mph_smoothed'] = df['haversine_smooth'] * 29 * 3600

Eric
Referee
ball
player1
player10
player11
player12
player13
player14
player15
player16
player17
player18
player19
player2
player20
player21
player22
player3
player4
player5
player6
player7
player8
player9


In [49]:
df.dropna()[~df['Name'].isin(["ball"])].sort_values('mph_smoothed')[-100:-50] #[df['Name'].isin(["ball"])]

/Users/ericstassen/opt/anaconda3/envs/vision37/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed,haversine_smooth_10,mph_smoothed_10
40175,998,1537.000000,348.500000,player5,H,"[41.70955397027558, -72.58548132818461]",41.709554,-72.585481,41.709553,-72.585483,0.000104,0.000164,10.899425,17.116541,0.000096,10.034628
6351,477,3757.238596,547.105965,player1,F,"[41.70896341407652, -72.58555776078096]",41.708963,-72.585558,41.708965,-72.585556,0.000118,0.000164,12.284544,17.134609,0.000187,19.501735
40173,996,1541.000000,351.500000,player5,H,"[41.70955219078671, -72.58548352376258]",41.709552,-72.585484,41.709552,-72.585484,0.000025,0.000164,2.613257,17.135874,0.000132,13.731702
40156,979,1570.500000,361.000000,player5,H,"[41.70954297470079, -72.58548978443194]",41.709543,-72.585490,41.709543,-72.585489,0.000085,0.000164,8.904671,17.165716,0.000248,25.841426
17398,1729,1280.000000,1164.000000,player14,F,"[41.70944403218361, -72.58585024696372]",41.709444,-72.585850,41.709445,-72.585845,0.000332,0.000165,34.665598,17.195720,0.000194,20.227855
17429,1760,1174.000000,1510.000000,player14,F,"[41.70942132961065, -72.58592531442736]",41.709421,-72.585925,41.709422,-72.585923,0.000156,0.000165,16.254970,17.213915,0.000168,17.500120
17430,1761,1173.500000,1523.000000,player14,F,"[41.709420227292625, -72.58592770723222]",41.709420,-72.585928,41.709421,-72.585925,0.000167,0.000165,17.423416,17.256721,0.000162,16.869014
6458,584,3794.961538,617.884615,player1,F,"[41.708968623378766, -72.58560857369356]",41.708969,-72.585609,41.708968,-72.585609,0.000032,0.000166,3.311559,17.283021,0.000032,3.298063
40172,995,1542.500000,352.000000,player5,H,"[41.70955171479397, -72.58548385685918]",41.709552,-72.585484,41.709551,-72.585485,0.000052,0.000166,5.387273,17.322891,0.000139,14.538705
17399,1730,1272.000000,1177.500000,player14,F,"[41.70944349114898, -72.585853676919]",41.709443,-72.585854,41.709444,-72.585850,0.000237,0.000166,24.769032,17.337368,0.000203,21.156382


Following code for distance to each other:

distance_df = []
for Frame, group in df.groupby("frame"):
    print(Frame)
    group['distances_mile'] = group.apply(lambda x: {"person_" + group.loc[index]['Name']: hs.haversine((x['lat'], x['lon']), (group.loc[index]['lat'], group.loc[index]['lon']), unit=Unit.MILES) for index in group.index}, axis=1)
    distance_df.append(group)
distance_df = pd.concat(distance_df)

In [50]:
distance_df = []
for Frame, group in df.groupby("frame"):
    print(Frame)
    group['ft_to_ball'] = group.apply(lambda x: hs.haversine((group.loc[group['Name'] == 'ball'].values[0][7], group.loc[group['Name'] == 'ball'].values[0][6]), (x['lat'], x['lon']), unit=Unit.MILES), axis=1) * 5280
    distance_df.append(group)
df = pd.concat(distance_df)

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278


In [51]:
df

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed,haversine_smooth_10,mph_smoothed_10,ft_to_ball
0,3,1510.0,449.5,Eric,H,"[41.70953107806433, -72.5855547689485]",41.709531,-72.585555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.555697
1959,3,3000.5,675.0,Referee,R,"[41.709182155140965, -72.58565954461588]",41.709182,-72.585660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.963785
3918,3,3806.5,583.5,ball,N,"[41.70895713163499, -72.5855838420091]",41.708957,-72.585584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
5877,3,3490.5,720.0,player1,F,"[41.7090677059074, -72.58567718713057]",41.709068,-72.585677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.130213
7836,3,2237.0,343.5,player10,F,"[41.709389662055564, -72.58544739029803]",41.709390,-72.585447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.615936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39179,1961,1746.5,1367.5,player4,H,"[41.70936509535353, -72.58590029866214]",41.709365,-72.585900,41.709365,-72.585901,0.000059,0.000025,6.129247,2.564292,0.000032,3.326721,68.514787
41138,1961,1351.0,427.5,player5,H,"[41.70956871400858, -72.58554465890536]",41.709569,-72.585545,41.709569,-72.585544,0.000022,0.000021,2.336735,2.158085,0.000022,2.325484,90.343949
43097,1961,1749.5,1087.5,player6,F,"[41.70938794743467, -72.58583042298297]",41.709388,-72.585830,41.709388,-72.585831,0.000010,0.000020,1.043236,2.041018,0.000010,1.035509,49.986264
45056,1961,3149.0,801.0,player7,H,"[41.70915327107922, -72.58572213679733]",41.709153,-72.585722,41.709153,-72.585721,0.000097,0.000088,10.082912,9.229999,0.000095,9.958884,12.569617


In [52]:
distance_df = []
for Name, group in df.groupby("Name"):
    print(Name)
    group['ft_traveled'] = group['haversine'].fillna(0).cumsum() * 5280
    distance_df.append(group)
df = pd.concat(distance_df).sort_values("frame")

Eric
Referee
ball
player1
player10
player11
player12
player13
player14
player15
player16
player17
player18
player19
player2
player20
player21
player22
player3
player4
player5
player6
player7
player8
player9


In [53]:
df.tail(50)

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed,haversine_smooth_10,mph_smoothed_10,ft_to_ball,ft_traveled
17628,1959,2107.0,1414.0,player14,F,"[41.70931618471511, -72.58591240130828]",41.709316,-72.585912,41.709317,-72.585912,0.000008,0.000016,0.849388,1.626972,0.000018,1.853143,68.126016,306.103508
33300,1959,812.0,558.0,player21,H,"[41.70962692563238, -72.58563249098452]",41.709627,-72.585632,41.709628,-72.585632,0.000036,0.000025,3.706575,2.643774,0.000025,2.561143,74.036287,338.277148
39177,1959,1740.5,1373.5,player4,H,"[41.709365419649714, -72.58590155901412]",41.709365,-72.585902,41.709366,-72.585902,0.000043,0.000023,4.537202,2.379337,0.000026,2.669411,67.078262,393.760670
7833,1959,3189.5,797.0,player1,F,"[41.70914425186741, -72.58571990757545]",41.709144,-72.585720,41.709145,-72.585718,0.000113,0.000076,11.846933,7.885097,0.000084,8.816615,12.590280,427.098368
19587,1959,831.5,729.0,player15,F,"[41.70957951311699, -72.58571014834985]",41.709580,-72.585710,41.709580,-72.585710,0.000021,0.000018,2.203316,1.889490,0.000022,2.296804,58.244527,421.000706
5874,1959,3600.5,852.5,ball,N,"[41.70905712151662, -72.58574251462765]",41.709057,-72.585743,41.709058,-72.585743,0.000056,0.000138,5.877083,14.372309,0.000072,7.538538,0.000000,980.925963
47014,1960,1511.5,389.5,player8,F,"[41.70954722193929, -72.5855132305703]",41.709547,-72.585513,41.709547,-72.585513,0.000002,0.000019,0.256003,1.986976,0.000026,2.664200,99.090186,396.373607
33301,1960,813.5,558.0,player21,H,"[41.70962668068645, -72.5856324664085]",41.709627,-72.585632,41.709627,-72.585632,0.000005,0.000024,0.557717,2.532553,0.000023,2.429476,74.037268,338.305354
7834,1960,3196.0,799.0,player1,F,"[41.709142930966316, -72.58572079514423]",41.709143,-72.585721,41.709144,-72.585720,0.000067,0.000074,7.008671,7.743923,0.000086,8.968031,12.229314,427.452829
25465,1960,3304.0,546.5,player18,F,"[41.70909698106541, -72.58557117728981]",41.709097,-72.585571,41.709099,-72.585571,0.000046,0.000071,4.773647,7.371842,0.000065,6.755413,62.237913,368.129138


In [54]:
df[df['ft_traveled'] == df['ft_traveled'].max()]

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed,haversine_smooth_10,mph_smoothed_10,ft_to_ball,ft_traveled
5876,1961,3623.5,842.5,ball,N,"[41.70905067001738, -72.58573776582354]",41.709051,-72.585738,41.709054,-72.585741,0.000256,0.000135,26.691717,14.07501,0.000098,10.220085,0.0,982.813873


### Plot to Pitch

In [55]:
quad_coords_out = {
    "lonlat": np.array([
        [41.709011, -72.585549], # Top Right
        [41.709691, -72.585688], # Top left
        [41.709500, -72.586105], # Bottom Left
        [41.709091, -72.586146] # Bottom Right
    ]),
    "pixel": np.array([
        [1540, 569], # top right
        [470, 569], # top left
        [666, 1299], # bottom left
        [1348, 1288] # bottom right
    ])
}
pm_out = PixelMapper(quad_coords_out["pixel"], quad_coords_out["lonlat"])
df['pixels_out'] = df.apply(lambda x: pm_out.lonlat_to_pixel((x['lon'], x['lat']))[0], axis=1)

In [56]:
df['x_out'] = df['pixels_out'].str[0]
df['y_out'] = df['pixels_out'].str[1]

In [57]:
df

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,...,haversine_smooth,mph,mph_smoothed,haversine_smooth_10,mph_smoothed_10,ft_to_ball,ft_traveled,pixels_out,x_out,y_out
0,3,1510.0,449.5,Eric,H,"[41.70953107806433, -72.5855547689485]",41.709531,-72.585555,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,63.555697,0.000000,"[850.2086690331566, 435.2805734722806]",850.208669,435.280573
3918,3,3806.5,583.5,ball,N,"[41.70895713163499, -72.5855838420091]",41.708957,-72.585584,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,"[1588.2450043486317, 615.7990228895604]",1588.245004,615.799023
29385,3,NaN,NaN,player20,H,"[nan, nan]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,"[nan, nan]",NaN,NaN
17631,3,NaN,NaN,player15,F,"[nan, nan]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,"[nan, nan]",NaN,NaN
25467,3,NaN,NaN,player19,F,"[nan, nan]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,"[nan, nan]",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15671,1961,533.5,430.0,player13,F,"[41.709715423733336, -72.58556759934096]",41.709715,-72.585568,41.709716,-72.585568,...,0.000009,0.198652,0.909337,0.000009,0.924594,95.506316,284.860922,"[479.94702919913135, 380.02873702906925]",479.947029,380.028737
3917,1961,1977.5,553.0,Referee,R,"[41.70941443657649, -72.58560813791131]",41.709414,-72.585608,41.709415,-72.585608,...,0.000024,2.615938,2.554981,0.000026,2.714609,61.755462,266.993168,"[1023.7263868871655, 540.9879587268434]",1023.726387,540.987959
45056,1961,3149.0,801.0,player7,H,"[41.70915327107922, -72.58572213679733]",41.709153,-72.585722,41.709153,-72.585721,...,0.000088,10.082912,9.229999,0.000095,9.958884,12.569617,463.935506,"[1354.4531825013821, 735.2604025488677]",1354.453183,735.260403
7835,1961,3207.5,800.0,player1,F,"[41.709140462458954, -72.58572116729795]",41.709140,-72.585721,41.709143,-72.585721,...,0.000071,5.966980,7.430322,0.000084,8.776878,11.522938,427.754608,"[1370.1341684919507, 735.9687184880422]",1370.134168,735.968718


In [58]:
import math

file_path = "".join(path.split(".")[:-1]).replace("labels/", "") + ".mov"  # change to your own video path
# print(file_path)
vid = cv2.VideoCapture(file_path)
vid_height = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
vid_width = vid.get(cv2.CAP_PROP_FRAME_WIDTH)
# print(vid_height, vid_width)

img = cv2.imread('pitch.png')
height,width,layers=img.shape
print(width, height, layers)

ratio = width / vid_width, height / vid_height
print(ratio)

video=cv2.VideoWriter("".join(path.split(".")[:-1]).replace("labels/", "") + "_analysis.mp4" ,cv2.VideoWriter_fourcc(*'mp4v'),out_fps,(width,height))
# print(video)

for j in df.frame.unique():
    img = cv2.imread('pitch.png')
    image = cv2.putText(img, str(j), (100,100), fontFace=2, fontScale=.8, color=(0, 0, 0), thickness=2, lineType = cv2.LINE_AA)
    # print(img)
    frame = df[df['frame'] == j].to_dict(orient='records')
    for position in frame:
        # print(position)
        #image = cv2.circle(img, (j*5,100), radius=10, color=(255, 0, 0), thickness=-1)
        if set(["Name", "x_out", "y_out", "Team"]) <= set(position.keys()):
            if position["Team"] == "F":
                color = (255, 0, 0)
            elif position["Team"] == "H":
                color = (0, 0, 255)
            elif position['Name'] == "ball":
                color = (0, 0, 0)
            else:
                color = (0, 0, 0)
            position['Name'] = position['Name'] if position['Name'] != "ball" else "*"
            if (not math.isnan(position['x_out'])) and (not math.isnan(position['y_out'])):
                if math.isnan(position['mph_smoothed']):
                    position['mph_smoothed'] = 0
                if math.isnan(position['ft_to_ball']):
                    position['ft_to_ball'] = 0
                # print("Writing Vid")
                image = cv2.putText(img, "."+position['Name'], (int(position['x_out']), int(position['y_out'])), fontFace=2, fontScale=.8, color=color, thickness=2, lineType = cv2.LINE_AA)
                image = cv2.putText(img, str(round(position['mph_smoothed'])) + "mph | " + str(round(position['ft_to_ball'])) + "ft | " + str(round(position['ft_traveled'])) + "ft", (int(position['x_out']), int(position['y_out'] + 20)), fontFace=2, fontScale=.5, color=color, thickness=1, lineType = cv2.LINE_AA)
                # cv2.imshow("Output", image)
                # if cv2.waitKey(1) & 0xFF == ord('q'):
                #     break

    video.write(img)

cv2.destroyAllWindows()
video.release()

1998 1412 3
(0.5203125, 0.6537037037037037)


In [59]:
df.to_csv("Hurling_intersquad_passes_final_data.pipe", sep="|", index=False)

def click_event(event, x, y, flags, params):
    object_frame, frame, frame_n = params
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        # displaying the coordinates on the Shell
        # print(x, ' ', y)
        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        # if os.environ["clicked_x"] and os.environ["clicked_y"]:
        #     print("Changing Frame back to original")
        object_frame = copy.deepcopy(frame)
        cv2.putText(object_frame, "." + str(x) + ',' +
                    str(y), (x,y), font,
                    1, (255, 0, 0), 2)
        cv2.imshow('Scraping feed', object_frame)
        os.environ["clicked_x"] = str(x)
        # print("X" + os.environ["clicked_x"] + " - " + str(x))
        os.environ["clicked_y"] = str(y)
        instructions_frame = copy.deepcopy(frame)
        cv2.putText(instructions_frame, "You selected: " + os.environ["clicked_x"] + ", " + os.environ["clicked_y"], (100,500), cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 0, 0), 5)
        cv2.imshow('Instructions feed', instructions_frame)
        print(frame_n, os.environ["clicked_x"], os.environ["clicked_y"])

import copy
import os

img = cv2.imread('pitch.png')
key_ind = "collecting"
while key_ind == "collecting":
    cv2.imshow('Data feed', img)
    cv2.setMouseCallback('Data feed', click_event, param=(img, img, 0))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
for i in range (1,5):
    cv2.waitKey(1)